In [ ]:
#Having data in multiple languages so using LASER embedding which helps to map the word with same meaning in multiple languages to the closest vector.
#using LASER embeddings even if data is less in each language the prediction model gives good accuracy.

In [ ]:
from laserembeddings import Laser
import pandas as pd
# input is the excel with columns('labels','filename','tokens','page_content','page_lang')
source = r"claim_doc.xlsx"
LANG = {"_lang_spanish":"es","_lang_english":"en","_lang_portuguese":"pt","_lang_dutch ":"nl","_lang_french":"fr","_lang_german":"de","_lang_italian":"it"}
data_df = pd.read_excel(source)
#lang = data_df.group_by('page_lang')

#Downloaded the laser model and referencing the path to files below
path_to_bpe_codes = "C:\Project\LASER_model\93langs.fcodes"
path_to_bpe_vocab = "C:\Project\LASER_model\93langs.fvocab"
path_to_encoder = r"C:\Project\LASER_model\bilstm.93langs.2018-12-26.pt"

laser = Laser(path_to_bpe_codes, path_to_bpe_vocab, path_to_encoder)

In [ ]:
data_df["labels"].value_counts()

In [ ]:
doctype_others = data_df[data_df["labels"] == "OTHERS"]
all_doc = data_df[data_df["labels"] != "OTHERS"]
print(doctype_others.shape)
print(all_doc.shape)
#print(df["labels"].value_counts())

In [ ]:
from sklearn.utils import resample
others_downsample = resample(doctype_others,
             replace=True,
             n_samples=1100,
             random_state=42)

print(others_downsample.shape)

In [ ]:
#data_downsampled = pd.concat([others_downsample,all_doc])
data_downsampled = pd.concat([others_downsample,all_doc])

print(data_downsampled["labels"].value_counts())

In [ ]:
temp = data_df.replace({'page_lang' : {"_lang_spanish":"es","_lang_english":"en","_lang_portuguese":"pt","_lang_dutch ":"nl","_lang_french":"fr","_lang_german":"de","_lang_italian":"it"}})
content = temp["page_content"]
#print("content",len(content))
lang_page = temp["page_lang"]

In [ ]:
# if all sentences are in the same language:
#lang_dict = []
#for k,v in LANG.items():
    #one_lang = data_df.loc[data_df['page_lang'] == k]
    #content = one_lang["page_content"]
    #lang_dict.append(content)
temp = data_df.replace({'page_lang' : {"_lang_spanish":"es","_lang_english":"en","_lang_portuguese":"pt","_lang_dutch ":"nl","_lang_french":"fr","_lang_german":"de","_lang_italian":"it"}})
content = temp["tokens"]
lang_page = temp["page_lang"]
#print(lang_page)
embeddings = laser.embed_sentences(content, lang=lang_page)  # lang is only used for tokenization
#embedded_dict[v] = embeddings
print("Embeddings:",embeddings)

# embeddings is a N*1024 (N = number of sentences) NumPy array

In [ ]:
# if all sentences are in the same language:
#embedded_dict = {}
for k,v in LANG.items():
    one_lang = data_df.loc[data_df['page_lang'] == k]
    content = one_lang["page_content"]
    embeddings = laser.embed_sentences(content, lang=v)  # lang is only used for tokenization
    embedded_dict[v] = embeddings
print("Embeddings:",embedded_dict)

# embeddings is a N*1024 (N = number of sentences) NumPy array

In [ ]:
import joblib
joblib.dump(embeddings, "embeddings_claim_doc.sav")

In [ ]:
import joblib
embeddings = joblib.load("embeddings_claim_doc.sav")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import resample
import pickle
import joblib

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=700,svd_solver="auto")
pca.fit(embeddings)
x = pca.transform(embeddings)

In [ ]:
with open("embeddings.txt",'w')as fp:
    fp.write(str(embeddings))

In [ ]:
#split datset into train, test dataset
X_train_1, X_test, y_train_1, y_test = train_test_split(embeddings, temp['labels'], test_size=0.2, random_state=0)

#Random oversampling and balancing the dataset
ros = RandomOverSampler(random_state=0)
ros.fit(X_train_1, y_train_1)
X_train, y_train = ros.fit_resample(X_train_1, y_train_1)

In [ ]:
#split datset into train, test dataset
X_train_1, X_test, y_train_1, y_test = train_test_split(embeddings, temp['labels'], test_size=0.2, random_state=0)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

model_svc = LinearSVC()
model = CalibratedClassifierCV(model_svc) 
model.fit(X_train_1, y_train_1)

In [ ]:
y_test.to_excel(r"pred_claim.xlsx")

In [ ]:
temp = pd.read_excel("pred_claim.xlsx")
filename_list = []
for val in temp['Number']:
    filename_index = data_df._get_value(val, 'filename')
    filename_list.append(filename_index)
l1 = pd.DataFrame(filename_list)
l1.columns = ['filename']
l1["Actual_Labels"] = temp['labels']
l1 ['predicted_Labels'] = lsvc_pred.tolist()
l1.to_excel("temp.xlsx", index=False)

In [ ]:
#saving the model
import joblib
joblib.dump(embeddings, "vectorizer_claim_doc.sav")
joblib.dump(model, "model_claim_doc.sav")

In [ ]:
#Logistic regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=0)
model.fit(X_train_1, y_train_1)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.fit(X_train_1, y_train_1)
print(classification_report(y_test,gradient_booster.predict(X_test)))

In [ ]:
gradient_pred = gradient_booster.predict(X_test)

In [ ]:
logistic_pred = model.predict(X_test)

# get the precision, recall, f1 score, accuracy scores

from sklearn.metrics import classification_report
print(classification_report(y_test,logistic_pred))

In [ ]:
lsvc_pred = model.predict(X_test)

# get the precision, recall, f1 score, accuracy scores

from sklearn.metrics import classification_report
print(classification_report(y_test,lsvc_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
conf_mat = confusion_matrix(y_test, lsvc_pred)
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(conf_mat, annot=True, fmt='d',xticklabels=['CLAIM','OTHERS'], yticklabels=['CLAIM','OTHERS'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()